In [1]:
import torch

In [2]:
x = torch.arange(12)
x

tensor([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11])

In [3]:
x.shape

torch.Size([12])

In [4]:
x.numel() # 元素总量

12

In [5]:
X = x.reshape(3, 4)
X

tensor([[ 0,  1,  2,  3],
        [ 4,  5,  6,  7],
        [ 8,  9, 10, 11]])

In [6]:
torch.zeros((2,3,4))

tensor([[[0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.]],

        [[0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.]]])

In [7]:
torch.ones((2,3,4))

tensor([[[1., 1., 1., 1.],
         [1., 1., 1., 1.],
         [1., 1., 1., 1.]],

        [[1., 1., 1., 1.],
         [1., 1., 1., 1.],
         [1., 1., 1., 1.]]])

In [10]:
torch.tensor(([2,1,4,3], [1,2,3,4], [4,3,2,1])).shape

torch.Size([3, 4])

### 常见的标准算数运算符（+、-、\*、/、和\**）都可以被升级为按元素运算

In [11]:
x = torch.tensor([1.0, 2, 4, 8])
y = torch.tensor([2,2,2,2])
x+y, x-y, x*y, x/y, x**y

(tensor([ 3.,  4.,  6., 10.]),
 tensor([-1.,  0.,  2.,  6.]),
 tensor([ 2.,  4.,  8., 16.]),
 tensor([0.5000, 1.0000, 2.0000, 4.0000]),
 tensor([ 1.,  4., 16., 64.]))

In [12]:
torch.exp(x)

tensor([2.7183e+00, 7.3891e+00, 5.4598e+01, 2.9810e+03])

In [14]:
X = torch.arange(12, dtype=torch.float32).reshape((3,4))
Y = torch.tensor([[2.0,1,4,3], [1,2,3,4], [4,3,2,1]])
torch.cat((X,Y), dim=0), torch.cat((X,Y), dim=1)

(tensor([[ 0.,  1.,  2.,  3.],
         [ 4.,  5.,  6.,  7.],
         [ 8.,  9., 10., 11.],
         [ 2.,  1.,  4.,  3.],
         [ 1.,  2.,  3.,  4.],
         [ 4.,  3.,  2.,  1.]]),
 tensor([[ 0.,  1.,  2.,  3.,  2.,  1.,  4.,  3.],
         [ 4.,  5.,  6.,  7.,  1.,  2.,  3.,  4.],
         [ 8.,  9., 10., 11.,  4.,  3.,  2.,  1.]]))

In [15]:
X == Y

tensor([[False,  True, False,  True],
        [False, False, False, False],
        [False, False, False, False]])

In [16]:
X.sum()

tensor(66.)

### 即使形状不同，我们仍然可以通过调用*广播机制*（broadcasting mechanism）来执行按元素操作

In [17]:
a = torch.arange(3).reshape((3,1))
b = torch.arange(2).reshape((1,2))
a, b

(tensor([[0],
         [1],
         [2]]),
 tensor([[0, 1]]))

In [18]:
a + b

tensor([[0, 1],
        [1, 2],
        [2, 3]])

In [ ]:
# 广播机制这也是容易出错的地方

In [19]:
X[-1], X[1:3]

(tensor([ 8.,  9., 10., 11.]),
 tensor([[ 4.,  5.,  6.,  7.],
         [ 8.,  9., 10., 11.]]))

In [21]:
X[1,2] = 9
X

tensor([[ 0.,  1.,  2.,  3.],
        [ 4.,  5.,  9.,  7.],
        [ 8.,  9., 10., 11.]])

In [23]:
X[0:2, :] = 12
X

tensor([[12., 12., 12., 12.],
        [12., 12., 12., 12.],
        [ 8.,  9., 10., 11.]])

### 运行一些操作可能会导致为新结果分配内存

In [24]:
before = id(Y)
Y = Y + X
id(Y) == before

False

In [25]:
# 执行原地操作
Z = torch.zeros_like(Y)
print('id(Z):', id(Z))
Z[:] = X + Y
print('idx(Z):', id(Z))

id(Z): 2157393775648
idx(Z): 2157393775648


In [26]:
# 如果在后续计算中没有重复使用X，我们也可以使用X[:] = X+Y或X += Y来减少操作的内存开销
before = id(X)
X += Y
id(X) == before

True

### 转换为NumPy张量

In [27]:
A = X.numpy()
B = torch.tensor(A)
type(A), type(B)

(numpy.ndarray, torch.Tensor)

### 将大小为1的张量转换为Python标量

In [28]:
a = torch.tensor([3.5])
a, a.item(), float(a), int(a)

(tensor([3.5000]), 3.5, 3.5, 3)

## Part 2：数据预处理

In [29]:
import os

In [49]:
# 手动创建csv文件并写入数据
os.makedirs(os.path.join('..', 'data'), exist_ok=True)
data_file = os.path.join('..', 'data', 'house_tiny.csv')
with open(data_file, 'w') as f:
    f.write('NumRooms, Alley, Price\n')
    f.write('NA, Pave, 127500\n')
    f.write('2, NA, 10600\n')
    f.write('4, NA, 178100\n')
    f.write('NA, NA, 140000\n')

In [50]:
# 从创建的csv文件中加载原始数据集
import pandas as pd

In [51]:
data = pd.read_csv(data_file)
data

,NumRooms,Alley,Price
0,NaN,Pave,127500
1,2.0,NA,10600
2,4.0,NA,178100
3,NaN,NA,140000


### 为了处理缺失的数据，典型的方法包括*插值*和*删除*，这里我们将考虑插值

In [56]:
inputs, outputs = data.iloc[:, 0:2], data.iloc[:, 2]
inputs.iloc[:,0] = inputs.iloc[:,0].fillna(inputs.iloc[:,0].mean())
print(inputs)

   NumRooms  Alley
0       3.0   Pave
1       2.0     NA
2       4.0     NA
3       3.0     NA


### 对于inputs中的类别值或离散值，我们将NaN视为一个类别

In [59]:
inputs = pd.get_dummies(inputs, dummy_na=True)
inputs

,NumRooms,Alley_ NA,Alley_ Pave,Alley_nan
0,3.0,0,1,0
1,2.0,1,0,0
2,4.0,1,0,0
3,3.0,1,0,0


In [60]:
# 转换为张量
X, y = torch.tensor(inputs.values), torch.tensor(outputs.values)
X, y

(tensor([[3., 0., 1., 0.],
         [2., 1., 0., 0.],
         [4., 1., 0., 0.],
         [3., 1., 0., 0.]], dtype=torch.float64),
 tensor([127500,  10600, 178100, 140000]))

### 对于深度学习一般用torch.float32浮点数，因为torch.float64浮点数计算慢

### reshape只改动形状，不能改变变量的地址，这里是容易出错的地方

In [61]:
# 看这个例子
a = torch.arange(12)
b = a.reshape((3,4))
b[:] = 2
a

tensor([2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])

In [62]:
# 上面的例子中，b改了把a也改了，所以不要这样操作